In [19]:
from datetime import datetime, timedelta
from pulp import LpProblem, LpVariable, lpSum, LpBinary, LpMinimize, LpStatus
import json

# Define the problem
prob = LpProblem("nurse_scheduling", LpMinimize)

# Parameters
start_date = datetime(2024, 1, 1)
num_days = 31
dates = [(start_date + timedelta(days=i)) for i in range(num_days)]
shifts = ["day", "evening", "night"]
nurse_ids = list(range(32))
max_shifts_per_month = 31
max_shifts_per_day = 2

# Define nurse rules
nurses = {}
for i in nurse_ids:
    if i < 1:  
        nurses[i] = ["CNL"]
    elif i < 6:  
        nurses[i] = ["Head", "Nurse"]
    else:  
        nurses[i] = ["Nurse"]

# Define shift demands 
demand = {
    "Saturday": {"day": {"Nurse": 6, "CNL": 1 , "Head": 0}, "evening": {"Nurse": 5, "CNL": 0, "Head": 1}, "night": {"Nurse": 5, "CNL": 0, "Head": 1}},
    "Sunday": {"day": {"Nurse": 6, "CNL": 1, "Head": 0}, "evening": {"Nurse": 5, "CNL": 0, "Head": 1}, "night": {"Nurse": 5, "CNL": 0, "Head": 1}},
    "Monday": {"day": {"Nurse": 6, "CNL": 1, "Head": 0}, "evening": {"Nurse": 5, "CNL": 0, "Head": 1}, "night": {"Nurse": 5, "CNL": 0, "Head": 1}},
    "Tuesday": {"day": {"Nurse": 6, "CNL": 1, "Head": 0}, "evening": {"Nurse": 5, "CNL": 0, "Head": 1}, "night": {"Nurse": 5, "CNL": 0, "Head": 1}},
    "Wednesday": {"day": {"Nurse": 6, "CNL": 1, "Head": 0}, "evening": {"Nurse": 5, "CNL": 0, "Head": 1}, "night": {"Nurse": 5, "CNL": 0, "Head": 1}},
    "Thursday": {"day": {"Nurse": 6, "CNL": 1, "Head": 0}, "evening": {"Nurse": 5, "CNL": 0, "Head": 1}, "night": {"Nurse": 5, "CNL": 0, "Head": 1}},
    "Friday": {"day": {"Nurse": 6, "CNL": 0, "Head": 1}, "evening": {"Nurse": 5, "CNL": 0, "Head": 1}, "night": {"Nurse": 5, "CNL": 0, "Head": 1}},
}

# Decision variables 
assignments = LpVariable.dicts(
    "Assignments",
    ((n, s, d, p) for n in nurse_ids for s in shifts for d in dates for p in nurses[n]),
    cat=LpBinary
)

demand_penalties = LpVariable.dicts(
    "demand penalties",
    ((d, s, p) for d in dates for s in shifts for p in ["Nurse", "CNL", "Head"]),
    lowBound=0, cat='Integer'
)
preferred_days = {
    1: [(dates[0], "evening"), (dates[0], "night")], 
    3: [(dates[1], "night")]   
}

off_days = {
    0: [(dates[0], "day"), (dates[1], "day")] 
}

# Objective function
penalty_weight = 1000
prob += lpSum(assignments) + penalty_weight * lpSum(demand_penalties)

# Constraints for shift demand
for d in dates:
    weekday = d.strftime("%A")
    for s in shifts:
        for p in demand[weekday][s]:
            required_count = demand[weekday][s][p]
            prob += (
                lpSum(assignments[n, s, d, p] for n in nurse_ids if p in nurses[n])
                + demand_penalties[d, s, p]
                >= required_count
            )

# Ensure at least one CNL or Head per shift
for d in dates:
    for s in shifts:
        prob += lpSum(assignments[n, s, d, "CNL"] for n in range(1)) + lpSum(assignments[n, s, d, "Head"] for n in range(1, 6)) >= 1

# Max shifts per month 
for n in nurse_ids:
    prob += lpSum(assignments[n, s, d, p] for s in shifts for d in dates for p in nurses[n]) <= max_shifts_per_month

# Max shifts per day 
for n in nurse_ids:
    for d in dates:
        prob += lpSum(assignments[n, s, d, p] for s in shifts for p in nurses[n]) <= max_shifts_per_day

# Ensure no night shift on the next day
for n in nurse_ids:
    for d in range(len(dates) - 1):
        prob += lpSum(assignments[n, 'day', dates[d + 1], p] + assignments[n, "night", dates[d], p] for p in nurses[n]) <= 1
        prob += lpSum(assignments[n, 'evening', dates[d + 1], p] + assignments[n, "night", dates[d], p] for p in nurses[n]) <= 1
        prob += lpSum(assignments[n, 'night', dates[d + 1], p] + assignments[n, "night", dates[d], p] for p in nurses[n]) <= 1

# Preferred days
for n, preferences in preferred_days.items():
        for d, s in preferences:
            prob += lpSum(assignments[n, s, d, p] for p in nurses[n]) == 1

# Off days 
for n, preferences in off_days.items():
    for d, s in preferences:
        prob += lpSum(assignments[n, s, d, p] for p in nurses[n]) == 0


# Solve the problem
prob.solve()

# Output results
def format_output_as_json(assignments, shifts, dates):
    output = []
    for d in dates:
        weekday = d.strftime("%A")
        for s in shifts:
            assigned_nurses = []
            for n in nurse_ids:
                for p in nurses[n]:
                    if assignments[n, s, d, p].varValue == 1:
                        assigned_nurses.append({"id": n, "role": p})
            if assigned_nurses:
                head_nurse = next((n for n in assigned_nurses if n["role"] == "Head"), None)
                cnl_nurse = next((n for n in assigned_nurses if n["role"] == "CNL"), None)
                output.append({
                    "date": d.strftime("%Y-%m-%d"),
                    "weekday": weekday,
                    "shift": s,
                    "nurses_assigned": assigned_nurses,
                    "head_nurse": head_nurse if head_nurse else {"id": None, "role": "None"},
                    "cnl_nurse": cnl_nurse if cnl_nurse else {"id": None, "role": "None"} if s == "day" and weekday != "Friday" else {"id": None, "role": "None"}
                })
    return json.dumps(output, indent=4)

# Generate JSON output
json_output = format_output_as_json(assignments, shifts, dates)

# Print output
print(json_output)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/miniconda3/envs/DS/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_z/mbltc11j1kxchc3w07g4ryr40000gn/T/6dcf3c11d5e74f649ff9d069547806c0-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_z/mbltc11j1kxchc3w07g4ryr40000gn/T/6dcf3c11d5e74f649ff9d069547806c0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4286 COLUMNS
At line 33275 RHS
At line 37557 BOUNDS
At line 41278 ENDATA
Problem MODEL has 4281 rows, 3720 columns and 17828 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2589 - 0.01 seconds
Cgl0002I 2 variables fixed
Cgl0003I 0 fixed, 184 tightened bounds, 958 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 148 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 sub

In [3]:
print("Status:", LpStatus[prob.status])


Status: Optimal
